# lg3 Fourier exogenous demo

Model a simple Fourier series using the same weekday from 1/2/3 weeks prior and compare to the target day.

In [195]:
import os
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- config ---
PROCESSED_ROOT = os.path.abspath(os.path.join('..', 'data', 'processed_sources'))
SOURCE = 'snu'  # elec1_f2 / ohsung_f2 / snu
SPLIT = 'train'  # train / val / test
TARGET_DATE = '2025-08-20'  # YYYY-MM-DD
COLUMN = 'Power'
FREQ = '5min'
FOURIER_K = 6
# number of steps per day (5min => 288)
FOURIER_S = 288
HOLIDAYS = {
    # '2025-09-15': 'Special Event',
}


In [196]:
def load_split(root, source, split):
    path = os.path.join(root, source, f'lg3_{split}.csv')
    df = pd.read_csv(path, parse_dates=[0], index_col=0)
    if COLUMN not in df.columns:
        raise ValueError(f'{COLUMN} not found in {path}')
    return df[[COLUMN]].sort_index()

def day_slice(df, day, freq):
    day_start = pd.to_datetime(day)
    day_end = day_start + timedelta(days=1) - timedelta(minutes=5)
    day_index = pd.date_range(day_start, day_end, freq=freq)
    out = df.loc[day_start:day_end].reindex(day_index)
    out[COLUMN] = out[COLUMN].interpolate(limit_direction='both')
    return out

df = load_split(PROCESSED_ROOT, SOURCE, SPLIT)
target_dt = datetime.strptime(TARGET_DATE, '%Y-%m-%d')
prior_dates = [target_dt - timedelta(days=7 * k) for k in (3, 2, 1)]

target_series = day_slice(df, target_dt, FREQ)
prior_series = [day_slice(df, d, FREQ) for d in prior_dates]

print('Target date:', target_dt.date())
print('Prior dates:', [d.date() for d in prior_dates])
for d in prior_dates + [target_dt]:
    label = HOLIDAYS.get(d.strftime('%Y-%m-%d'))
    if label:
        print(f'Holiday: {d.date()} -> {label}')

# quick head/tail preview for each day
for label, series in zip(['3w', '2w', '1w', 'target'], prior_series + [target_series]):
    print(f'[{label}] head:', series.head(3))
    print(f'[{label}] tail:', series.tail(3))


Target date: 2025-08-20
Prior dates: [datetime.date(2025, 7, 30), datetime.date(2025, 8, 6), datetime.date(2025, 8, 13)]
[3w] head:                       Power
2025-07-30 00:00:00  1.0998
2025-07-30 00:05:00  1.2458
2025-07-30 00:10:00  3.3540
[3w] tail:                       Power
2025-07-30 23:45:00  2.0016
2025-07-30 23:50:00  2.4796
2025-07-30 23:55:00  2.2000
[2w] head:                       Power
2025-08-06 00:00:00  2.6394
2025-08-06 00:05:00  2.7324
2025-08-06 00:10:00  2.8960
[2w] tail:                       Power
2025-08-06 23:45:00  3.5280
2025-08-06 23:50:00  3.4420
2025-08-06 23:55:00  3.4332
[1w] head:                       Power
2025-08-13 00:00:00  2.0134
2025-08-13 00:05:00  1.6460
2025-08-13 00:10:00  1.9088
[1w] tail:                       Power
2025-08-13 23:45:00  1.4370
2025-08-13 23:50:00  1.4722
2025-08-13 23:55:00  2.0682
[target] head:                       Power
2025-08-20 00:00:00  1.6632
2025-08-20 00:05:00  1.3862
2025-08-20 00:10:00  1.4880
[target] tail:

In [197]:
# ============================================================
# 2) FFT-based Fourier (weighted prior weeks only)
# ============================================================

def fft_reconstruct(y: np.ndarray, K: int):
    y = np.asarray(y, dtype=float).reshape(-1)
    n = len(y)
    fft = np.fft.rfft(y)

    # keep DC + top-K frequency components by magnitude
    keep = np.zeros_like(fft)
    keep[0] = fft[0]

    if K > 0 and len(fft) > 1:
        mags = np.abs(fft)
        idx = np.argsort(mags[1:])[-K:] + 1
        keep[idx] = fft[idx]

    recon = np.fft.irfft(keep, n=n)
    return recon

# weighted average of 3 prior weeks (same weekday)
weights = np.array([0.1, 0.3, 0.6], dtype=float)  # 3w, 2w, 1w
weights = weights / weights.sum()
prior_matrix = np.vstack([s[COLUMN].values for s in prior_series])
weighted_prior = (prior_matrix.T @ weights).T

# Fourier reconstruction for one day (from prior weeks only)
fourier_day = fft_reconstruct(weighted_prior, K=FOURIER_K)

# predict for 4 days by repeating the same Fourier day
all_series = prior_series + [target_series]
concat_values = np.concatenate([s[COLUMN].values for s in all_series])

day_steps = len(target_series)
total_steps = day_steps * 4
fourier_all = np.concatenate([fourier_day] * 4)

# quick sanity checks
for label, series in zip(['3w', '2w', '1w', 'target'], all_series):
    nan_count = np.isnan(series[COLUMN].values).sum()
    print(f'[INFO] {label} NaNs: {nan_count} / {len(series)}')


[INFO] 3w NaNs: 0 / 288
[INFO] 2w NaNs: 0 / 288
[INFO] 1w NaNs: 0 / 288
[INFO] target NaNs: 0 / 288


In [198]:
# compute error on target segment (use one-day Fourier)
target_pred = fourier_day
if len(target_pred) != len(target_series):
    raise ValueError('fourier_day length mismatch with target series')

target_true = target_series[COLUMN].values
mse = np.mean((target_true - target_pred) ** 2)
mae = np.mean(np.abs(target_true - target_pred))
print(f'[INFO] Target MSE: {mse:.6f} | MAE: {mae:.6f}')


[INFO] Target MSE: 0.706919 | MAE: 0.725198


day_steps: 288 total_steps: 1152
concat_values len: 1152 fourier len: 1152


In [199]:
# debug lengths
print('day_steps:', day_steps, 'total_steps:', total_steps)
print('concat_values len:', len(concat_values), 'fourier len:', len(fourier_all))

labels = [f'3w prior {prior_dates[0].date()}', f'2w prior {prior_dates[1].date()}', f'1w prior {prior_dates[2].date()}', f'target {target_dt.date()}']

# build a continuous 4-day index starting from the oldest prior day
start_day = prior_dates[0].date()
continuous_index = pd.date_range(start=pd.to_datetime(start_day), periods=total_steps, freq=FREQ)

plt.figure(figsize=(14, 5))

# plot the 4 concatenated days as colored segments
for i, (color, label) in enumerate(zip(['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'], labels)):
    start = i * day_steps
    end = (i + 1) * day_steps
    plt.plot(continuous_index[start:end], concat_values[start:end], color=color, linewidth=0.9, label=label)

# overlay Fourier prediction
plt.plot(continuous_index, fourier_all, color='black', linewidth=1.2, label='fourier')

# mark day boundaries with date labels
boundary_positions = [continuous_index[i * day_steps] for i in range(4)]
boundary_labels = [d.strftime('%Y-%m-%d') for d in prior_dates + [target_dt]]
plt.xticks(boundary_positions, boundary_labels, rotation=0)

plt.title(f'{SOURCE} {SPLIT} {COLUMN} | 3 prior weeks + target (continuous)')
plt.xlabel('Date')
plt.ylabel(COLUMN)
plt.legend()
plt.tight_layout()
plt.show()
